In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
sns.set(style="darkgrid")

In [ ]:
df1 = pd.read_csv('../input/diabetes-dataset/diabetes2.csv')
df1.shape

* We have total 767 rows and 9 columnes and looking at the Dataframe info it seems their is no NA, NAN records present in dataframe
* if we look at the snapshot of the dataframe above we observe zero's (0) value oresent in some of the columns

In [ ]:
df1.info()

In [ ]:
df1.describe()

#### Many column seems to have min value zero let us observe the Proportion ok zero's in each column


In [ ]:

print("Proportion of missing values")
missing_percentage = (df1==0).sum()*100/df1.shape[0]
missing_percentage

#### let us observe if their is any dependency of missing values in skinfold_thickness and other columns


In [ ]:
bp_df = df1.loc[df1['SkinThickness']==0]
print("Count of zeros in blood_pressure:", (bp_df['BloodPressure']==0).sum())
print("Count of zeros in skinfold_thickness:", (bp_df['SkinThickness']==0).sum())
print("Count of zeros in insulin:", (bp_df['Insulin']==0).sum())

#### let us observe if their is any dependency of missing values in insulin and other columns


In [ ]:
bp_df = df1.loc[df1['Insulin']==0]
print("Count of zeros in blood_pressure:", (bp_df['BloodPressure']==0).sum())
print("Count of zeros in skinfold_thickness:", (bp_df['SkinThickness']==0).sum())
print("Count of zeros in insulin:", (bp_df['Insulin']==0).sum())

## Missing Value :
A important point hear, zeros in Pragnancy column indicates a feature which could not considered as Missing Value rather zeros present in rest of the columns are to considered as a missing value and hence we will go ahead replacing zeros to Nan.*
Count of zeros in "blood_pressure" column is 35 which is seems to have dependency on other column like Insulin. Reason could be a User had not attended the Test post 2 hour of lunch and hence we are observing Missing at random data in the perticular column along with other column - skinfold_thickness & insulin.
Again from the above result we can observe that, missing values in "skinfold_thickness" is correlated/dependent on "insulin" column as missing records of 227 and it is same.
Treatment to the Missing values are either ::
Drop the records
Mean/ Median/ Mode Imputation
Using model to predict missing values.

In [ ]:
m_col = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
colnum=1
for col in m_col:
    df1[col]=df1[col].replace(0,np.nan)
    colnum+=1
df1.isnull().sum()/df1.shape[0]


* ##### Missing records in insulin, skinfthickness are almost 50%, 30% respectively we can not dorp the the column.
* ##### Now that we have Null values in the columns- we can now impute the missing values using KNN Imputer. 

###  Load KNNImputer

In [ ]:
import sklearn
from sklearn.impute import KNNImputer,SimpleImputer

In [ ]:
imputer = KNNImputer(n_neighbors=14)
k_df1=imputer.fit_transform(df1)
df=pd.DataFrame(k_df1,columns=df1.columns)
df.shape

In [ ]:
df.isnull().sum()

### Now that we have missing values replaced using model we can go ahead analyse different columns and their patterns 

In [ ]:
df['Outcome'].value_counts().plot.pie()
df['Outcome'].value_counts(normalize=True)

#### Around 65% of users are healthy ( non-Diabetic)

In [ ]:

N_col = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
plt.figure(figsize=(20,60), facecolor='white')
plotnum=1
for col in N_col:
    ax=plt.subplot(9,3,plotnum)
    sns.boxplot(y=df[col],x=df['Outcome'])
    plt.title(col)
    plotnum+=1
plt.show()

In [ ]:
plt.figure(figsize=(20,60), facecolor='white')
plotnum=1
for col in N_col:
    ax=plt.subplot(9,3,plotnum)
    sns.distplot(df[col], bins=20)
    plt.title(col)
    plotnum+=1
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df.corr(),annot=True, cmap='Reds',center = 0.4)
plt.title('Correlation of Different columns in dataframe')
plt.show()

### glucose and BMI and Age shows stronger relationship with Outcome.

# Let's create a Data Model

In [ ]:
X=df.drop(['Outcome'],axis=1)
y=df.Outcome


#### Splitting DataSet into Train and Test:

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score, roc_curve,auc, plot_confusion_matrix

#### let us use Logistic Regression with Liblinear Solver

In [ ]:

log_model = LogisticRegression(solver='liblinear')
log_model.fit(X_train,y_train)
y_pred = log_model.predict(X_test)

In [ ]:
res= confusion_matrix(y_test, y_pred)
sns.heatmap(res/np.sum(res), annot=True, fmt='.2%')
plt.xlabel('Actual label')
plt.ylabel('Predicted label')
plt.show()

In [ ]:
def Model_Performance(test,pred):
    precision = precision_score(test,pred)
    recall = recall_score(test,pred)
    f1 = f1_score(test,pred)
    print('1. Confusion Matrix:\n',confusion_matrix(test, pred))
    print("\n2. Accuracy Score:", round(accuracy_score(test, pred)*100,2),"%")
    print("3. Precision:", round(precision*100,2),"%")
    print("4. Recall:",round(recall*100,2),"%" )
    print("5. F1 Score:",round(f1*100,2),"%" )
    print("6. clasification report:\n",classification_report(test, pred))

In [ ]:
Model_Performance(y_test, y_pred)

In [ ]:
fpr,tpr,_=roc_curve(y_test,y_pred)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,label='ROC curve(area= %2.f)' %roc_auc)
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
print( 'AUC Score of Model:',round(roc_auc_score(y_test,y_pred)*100,2),"%")

In [ ]:
model_f=pd.DataFrame()
model_f['Features'] = list(X_train.columns)
model_f['importance'] = list(log_model.coef_[0])

In [ ]:
imp_check=pd.DataFrame(model_f.sort_values(by='importance'))
imp_check.plot.barh(x='Features',y='importance',title='Features by Importance')
plt.show()


#### Feataures by Importance in our model

In [ ]:
print(imp_check)